In [1]:
#Installing the libraries for splitting up the folders
!pip install split-folders -q

In [7]:
!pip install tensorflow -q

In [20]:
#Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import splitfolders
import os
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report
import seaborn as sns
print("All libraries are installed and imported succesfully")

All libraries are installed and imported succesfully


In [18]:
#Set up Kaggle API Key
if not os.path.exists('/root/.kaggle/kaggle.json'):
    print("Please upload your 'kaggle.json' file.")
    uploaded = files.upload()
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
else:
    print("APi key already exists.")

# Download and unzip the dataset if it doesn't exist
raw_data_dir = '/content/Citrus_Leaves_Raw'
if not os.path.exists(raw_data_dir):
    print("Downloading and unzipping the Citrus Leaf dataset...")
    !kaggle datasets download -d myprojectdictionary/citrus-leaf-disease-image -p /content
    !unzip -q /content/citrus-leaf-disease-image.zip -d {raw_data_dir}
    print("✅ Dataset is ready.")
else:
    print("Dataset already downloaded.")

APi key already exists.
Dataset already downloaded.


In [19]:
#spilt data into train, val and test set folder
input_folder = '/content/Citrus_Leaves_Raw/Citrus Leaf Disease Image'
output_folder = '/content/Citrus_Leaves_Split'
if not os.path.exists(output_folder):
    print("Splitting data into train, val, and test sets...")
    splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.7, .15, .15))
    print("data splitting complete")
else:
    print("Data already split.")

Data already split.


In [24]:
#Defining the directories and image parameters
train_dir = os.path.join(output_folder, 'train')
val_dir = os.path.join(output_folder, 'val')
test_dir = os.path.join(output_folder, 'test')
Image_height = 224
Image_width = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(Image_height, Image_width), batch_size=batch_size, class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    val_dir, target_size=(Image_height, Image_width), batch_size=batch_size, class_mode='categorical')

test_generator = train_datagen.flow_from_directory(
    test_dir, target_size=(Image_height, Image_width), batch_size=batch_size, class_mode='categorical')

print("Data generators are ready.")
print("Class Indices", test_generator.class_indices)

Found 423 images belonging to 5 classes.
Found 88 images belonging to 5 classes.
Found 96 images belonging to 5 classes.
Data generators are ready.
Class Indices {'Black spot': 0, 'Canker': 1, 'Greening': 2, 'Healthy': 3, 'Melanose': 4}
